## PEC Automated Machine Learning

El objetivo de este notebook es generar un modelo de machine learning a través de la librería auto-sklearn para ver el funcionamiento del aprendizaje automático automatizado por parte de esta librería y compararlo posteriormente con un modelo generado a través de la herramienta Dataiku.

Para ello se hará uso del dataset Telco_Churn_Customer, que recoge datos sobre clientes telco y el objetivo es poder predecir con el mayor acierto posible si un cliente cambiará de empresa o no en base a los datos del dataset. El cambio de empresa se recoge en la variable Churn, que será el target.

En primer lugar se importan las librerías:

In [57]:
import sklearn
import pandas as pd
import autosklearn.classification

Se carga el dataset. Del mismo ya se conoce la existencia de valores nulos en una de las variables ya que la herramienta Dataiku lo detectó, y será necesario tratarlo.

In [41]:
df = pd.read_csv("Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [42]:
# Se comprueba el tamaño del dataset
df.shape

(7043, 21)

In [43]:
# Se comprueban los tipos de las columnas
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Sorprende que la columna TotalCharges, la cuál contenía valores vacíos, se ha detectado como de tipo object, en vez de tipo numérico. Se convierte dicha columna a tipo numérico.

In [44]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors = 'coerce')

Al aplicar el cambio de tipo sin especificar el atributo errors = 'coerce', resultaba en el error:

ValueError: Unable to parse string " "

Por lo tanto, parece ser que los valores nulos del dataset que en Dataiku se habían detectado automáticamente, al cargar el dataset a través de pandas, se han leído como espacios blancos, que no pueden ser parseados a números, por lo que hay que incluir el atributo errors para convertirlo en NaN.

In [45]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

De los 7043 registros hay 11 que contienen valor para la variable TotalCharges, por lo que se eliminan.

In [46]:
df.dropna(inplace=True)
df.shape

(7032, 21)

Una vez el dataset está listo, se pueden modificar los tipos de las columnas. Para ello, en primer lugar se pueden imprimir los valores de cada variable:

In [47]:
for i in df.columns:
    print("Los valores únicos para la variable {} son: {}\n".format(i, df[i].unique()))

Los valores únicos para la variable customerID son: ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

Los valores únicos para la variable gender son: ['Female' 'Male']

Los valores únicos para la variable SeniorCitizen son: [0 1]

Los valores únicos para la variable Partner son: ['Yes' 'No']

Los valores únicos para la variable Dependents son: ['No' 'Yes']

Los valores únicos para la variable tenure son: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]

Los valores únicos para la variable PhoneService son: ['No' 'Yes']

Los valores únicos para la variable MultipleLines son: ['No phone service' 'No' 'Yes']

Los valores únicos para la variable InternetService son: ['DSL' 'Fiber optic' 'No']

Los valores únicos para la variable OnlineSecurity son: ['No' 'Yes' 'No internet service']

Lo

Se elimina la variable customerID ya que no es necesaria para el modelo.

In [48]:
df.drop('customerID', axis = 1, inplace = True)

Se modifican los tipos del resto de variables.

In [49]:
desired_boolean_columns = ['gender']
desired_categorical_columns = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                               'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod', 'SeniorCitizen', 
                               'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
desired_numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']

for column in df.columns:
    if column in desired_boolean_columns:
        df[column] = df[column].astype('bool')
    elif column in desired_categorical_columns:
        df[column] = df[column].astype('category')
    else:
        df[column] = pd.to_numeric(df[column])

In [50]:
# Se comprueba el cambio
df.dtypes

gender                  bool
SeniorCitizen       category
Partner             category
Dependents          category
tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float64
Churn               category
dtype: object

Una vez el dataset está preparado, se puede dividir entre entrenamiento y test

In [58]:
y = df['Churn'].copy()
X = df[df.columns[0:-1]].copy()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)

Se va a realizar un primer entrenamiento con un límite de tiempo bajo:

In [93]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=10,
    tmp_folder='/tmp/autosklearn_classification_pec10_tmp',
    output_folder='/tmp/autosklearn_classification_pec10_out',
    delete_tmp_folder_after_terminate=True, 
    delete_output_folder_after_terminate=True
)

In [94]:
automl.fit(X_train, y_train, dataset_name='Telco-Customer-Churn')

AutoSklearnClassifier(output_folder='/tmp/autosklearn_classification_pec10_out',
                      per_run_time_limit=10, time_left_for_this_task=120,
                      tmp_folder='/tmp/autosklearn_classification_pec10_tmp')

Una vez entrenado, se imprimen por pantalla los modelos así como las estadísticas del entrenamiento.

In [95]:
print(automl.show_models())

[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 1, 'classifier:random_forest:min_samples_split': 2, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_preprocessing:categorical_tran

In [96]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: Telco-Customer-Churn
  Metric: accuracy
  Best validation score: 0.788627
  Number of target algorithm runs: 13
  Number of successful target algorithm runs: 4
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0



Se observa como las restricciones de tiempo conllevan que muchos de los algortimos no se hayan podido entrenar satisfactoriamente. El algoritmo seleccionado en este caso ha sido simplemente el Random Forest.

A continuación se calculan las métricas con las que se va a analizar el modelo:

In [97]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
print("F1 Score score:", sklearn.metrics.f1_score(y_test, predictions, pos_label = 'Yes'))

Accuracy score: 0.7770193401592719
F1 Score score: 0.5288461538461539


Vemos que los resultados son mejorables.
Se hace una segunda prueba con los valores por defecto para los atributos time_left_for_this_task y per_run_time_limit, que se incrementan considerablemente por lo que el rendimiento del modelo final debería mejorar.

In [101]:
automl2 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=360,
    tmp_folder='/tmp/autosklearn_classification_pec36_tmp',
    output_folder='/tmp/autosklearn_classification_pec36_out',
    delete_tmp_folder_after_terminate=True, 
    delete_output_folder_after_terminate=True
)

In [102]:
automl2.fit(X_train, y_train, dataset_name='Telco-Customer-Churn')

AutoSklearnClassifier(output_folder='/tmp/autosklearn_classification_pec36_out',
                      per_run_time_limit=360,
                      tmp_folder='/tmp/autosklearn_classification_pec36_tmp')

In [103]:
print(automl2.show_models())

[(0.780000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'polynomial', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 0.025032327219520423, 'classifier:adaboost:max_depth': 6, 'classifier:adaboost:n_estimators': 124, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:n_quantiles': 950, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:output_distribution': 'normal', 'feature_preprocessor:polynomial:degree': 2, 'feature_preprocessor:polynomial:include_bias': 'True', 'featur

In [104]:
print(automl2.sprint_statistics())

auto-sklearn results:
  Dataset name: Telco-Customer-Churn
  Metric: accuracy
  Best validation score: 0.816772
  Number of target algorithm runs: 17
  Number of successful target algorithm runs: 8
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0



Se observa como se han podido probar un mayor número de algoritmos, aunque nuevamente muchos de ellos han excedido el tiempo límite. Se comprueba como el tiempo es una gran barrera y es necesario incrementarlo para poder conseguir entrenar satisfactoriamente un mayor número de algortimos.

En este caso se observa como el resultado final es un algortimo que es un ensemble de varios.

Finalmente se analiza el rendimiento del modelo.

In [105]:
predictions = automl2.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
print("F1 Score score:", sklearn.metrics.f1_score(y_test, predictions, pos_label = 'Yes'))

Accuracy score: 0.79806598407281
F1 Score score: 0.5523329129886507


 Se observa como el rendimiento del modelo ha mejorado, pero es presumible que un incremento del tiempo de ejecución permitiría más pruebas que conllevarían mejores resultados finales.